In [1]:
import os
import subprocess
import pandas as pd
import json
import pycocotools

import viame2coco.viame2coco as viame2coco

In [2]:
imagery_raw_bucket = "amlr-gliders-imagery-raw-dev"
imagery_raw_mt = "/home/sam_woodman_noaa_gov/amlr-gliders-imagery-raw-dev"

csv_base_path = "/home/sam_woodman_noaa_gov/jw-annotations/amlr08-20220513"

In [3]:
# Mount bucket(s)
cmd = ["gcsfuse", "--implicit-dirs", imagery_raw_bucket, imagery_raw_mt, "-o", "ro"]
subprocess.run(cmd)

{"timestamp":{"seconds":1743037195,"nanos":683903000},"severity":"INFO","message":"Start gcsfuse/2.9.0 (Go version go1.23.5) for app \"\" using mount point: /home/sam_woodman_noaa_gov/amlr-gliders-imagery-raw-dev\n"}
{"timestamp":{"seconds":1743037195,"nanos":683933964},"severity":"INFO","message":"GCSFuse config","config":{"AppName":"","CacheDir":"","Debug":{"ExitOnInvariantViolation":false,"Fuse":false,"Gcs":false,"LogMutex":false},"EnableAtomicRenameObject":false,"EnableHns":true,"FileCache":{"CacheFileForRangeRead":false,"DownloadChunkSizeMb":50,"EnableCrc":false,"EnableODirect":false,"EnableParallelDownloads":false,"MaxParallelDownloads":16,"MaxSizeMb":-1,"ParallelDownloadsPerFile":16,"WriteBufferSize":4194304},"FileSystem":{"DirMode":"755","DisableParallelDirops":false,"FileMode":"644","FuseOptions":["ro"],"Gid":-1,"HandleSigterm":true,"IgnoreInterrupts":true,"KernelListCacheTtlSecs":0,"PreconditionErrors":true,"RenameDirLimit":0,"TempDir":"","Uid":-1},"Foreground":false,"GcsAuth

Error: daemonize.Run: readFromProcess: sub-process: Error while mounting gcsfuse: mountWithArgs: mountWithStorageHandle: mount: mount: running /usr/bin/fusermount: exit status 1


CompletedProcess(args=['gcsfuse', '--implicit-dirs', 'amlr-gliders-imagery-raw-dev', '/home/sam_woodman_noaa_gov/amlr-gliders-imagery-raw-dev', '-o', 'ro'], returncode=1)

{"timestamp":{"seconds":1743037195,"nanos":864836516},"severity":"INFO","message":"2025/03/27 00:59:55.864814 Error occurred during command execution: daemonize.Run: readFromProcess: sub-process: Error while mounting gcsfuse: mountWithArgs: mountWithStorageHandle: mount: mount: running /usr/bin/fusermount: exit status 1"}


## Convert VIAME CSV file to COCO format

In [4]:
# Read in CSV file(s)
os.listdir(csv_base_path)

csv_path2 = os.path.join(csv_base_path, 'amlr08-20220513-dir0002-annotations-manual.csv')

d2 = pd.read_csv(csv_path2)
display(d2)

display(d2.iloc[1:, 9].value_counts())

# pref = "/home/sam_woodman_noaa_gov/Dir0002"
# d2.loc[1:, '2: Video or Image Identifier'] = (
#     [f"{pref}/{i}" for i in d2['2: Video or Image Identifier'].values[1:]])
# d2.to_csv("tmp2.csv", index=False)

,# 1: Detection or Track-id,2: Video or Image Identifier,3: Unique Frame Identifier,4-7: Img-bbox(TL_x,TL_y,BR_x,BR_y),8: Detection or Length Confidence,9: Target Length (0 or -1 if invalid),10-11+: Repeated Species,Confidence Pairs or Attributes
0,# metadata,"exported_by: ""dive:python""","exported_time: ""Mon Dec 9 23:56:55 2024""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,SG01 20220513-205705-001.jpg,0,1719.0,502.0,1787.0,595.0,1.0,-1.0,copepod,1.0
2,1,SG01 20220513-205736-011.jpg,10,1916.0,1893.0,2016.0,1988.0,1.0,-1.0,copepod,1.0
3,2,SG01 20220513-205739-012.jpg,11,2050.0,1107.0,2166.0,1257.0,1.0,-1.0,copepod,1.0
4,3,SG01 20220513-205742-013.jpg,12,3199.0,1578.0,3335.0,1719.0,1.0,-1.0,copepod,1.0
...,...,...,...,...,...,...,...,...,...,...,...
456,455,SG01 20220513-214016-018.jpg,757,1360.0,2141.0,1458.0,2280.0,1.0,-1.0,copepod,1.0
457,456,SG01 20220513-214019-019.jpg,758,1297.0,2318.0,1487.0,2504.0,1.0,-1.0,hydrozoan,1.0
458,457,SG01 20220513-214019-019.jpg,758,2419.0,2061.0,2538.0,2165.0,1.0,-1.0,copepod,1.0
459,458,SG01 20220513-214022-020.jpg,759,1912.0,1827.0,2009.0,1962.0,1.0,-1.0,copepod,1.0


10-11+: Repeated Species
copepod         164
radiolarian     163
larvacean        35
unid             34
chaetognath      19
hydrozoan        15
siphonophore     11
ctenophore        6
fish              4
egg               4
echinoderm        2
pteropod          2
euphausiid        1
Name: count, dtype: int64

In [5]:
# viame2coco.viame2coco(csv_path2, "temporary description")
dir_path = os.path.join(
        f"/home/sam_woodman_noaa_gov/{imagery_raw_bucket}", 
        "SANDIEGO/2022/amlr08-20220513/images/Dir0002")

config = {
    'filename': 1,
    'label': 9, 
    'bbox_tlbr': {
        'tlx': 3,
        'tly': 4,
        'brx': 5,
        'bry': 6
    }
}

c2 = viame2coco.viame2coco(
    csv_path2, "temporary description", 
    filename_base=dir_path, config=config
)
display(c2)

j2 = c2.to_json("../tmp2-coco.json")
display(j2)


None

### Explore COCO output

In [ ]:
with open("../tmp2-coco.json") as fin:
    j2b = json.load(fin)

print(j2b.keys())

j2b_images = j2b["images"]
print(j2b_images)

j2b_anno = j2b["annotations"]
print(j2b_anno)

# print(j2b["info"])
# print(j2b["licenses"])
# print(j2b["categories"])

# j2b_cat = j2b["categories"]
# j2b_cat

## Extract regions from images
 
Using COCO output

In [ ]:
# with open("../tmp2-coco.json") as fin:
#     c2 = json.load(fin)
# c2.info.year

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
import math


# #%% Function to see if one bbox (embryo) is inside another (worm)
# def bbox_relation(wormbbox, embryobbox):
#     #if worm x (& y) is smaller than embryo x (&y), 
#     if wormbbox[0] <= embryobbox[0] and wormbbox[1] <= embryobbox[1]:
#         #If worm xmax is bigger than embryo xmax
#         if (wormbbox[0] + wormbbox[2]) > (embryobbox[0] + embryobbox[2]):
#             #If worm ymax is bigger than embryo ymax
#             if (wormbbox[1] + wormbbox[3]) > (embryobbox[1] + embryobbox[3]):
#                 inside = True
#             else:
#                 inside = False
#         else:
#             inside = False
#     else:
#         inside = False
#     return(inside)

# #%% Function to transpose bbox to cropped image with no padding
# def bbox_transpose(wormbbox, embryobbox):
#     embryobbox[0] = embryobbox[0] - wormbbox[0]#x = x-xmin
#     embryobbox[1] = embryobbox[1] - wormbbox[1]#y = y-ymin
    
#     return(embryobbox)

# #%%Function to transpose seg to cropped image with no padding
# def seg_transpose(wormbbox, embryo_seg):
#     seg_holder = embryo_seg
#     for y in range(0, len(seg_holder[0])): #y is a single point within the segmentation coords
#         if y % 2 == 0:#if y is x coord
#             seg_holder[0][y] -=wormbbox[0] 
#         elif y % 2 == 1:#if y is y cord
#             seg_holder[0][y] -=wormbbox[1]
#     return(seg_holder)


#%%
'''
Big loop to take input images - crop the worm bboxes from them, 
and transpose worm and embryo annotations to these new cropped images.
Annotation ID duplicates are not yet dealt with, 
presumably just aother loop at the end of things will sort it out

Currently, I think it only needs working dir and annFile as variables,
- possible to make into a function returning the dump dictionary?
'''

#Directory where files will be dumped, and where imput images are
working_dir = dir_path
working_cat = 'copepod'
#Annotation file for images
annFile = "../tmp2-coco.json"
#dump dict looks like a java object, but it really a python dictionary we're going to be saving at the end
dump_dict = {"image": [], "categories": [], "annotations": [], }
coco=COCO(annFile) #load via cocoapi
cats = coco.loadCats(coco.getCatIds()) #get categorys to add to each json out
cats_names = [i["name"] for i in cats]
dump_dict["categories"] = cats



catIds = coco.getCatIds(catNms=[working_cat]) #use to pull cat ID from cat
imgIds = coco.getImgIds(catIds=catIds ) #get all images with the specified category above in
new_im_id = 0



In [ ]:
### Sam exp - loop thorugh

from pycocotools.coco import COCO
import cv2
import json
# import matplotlib.pyplot as plt
import math

#Directory where files will be dumped, and where imput images are
# working_dir = dir_path
# working_cat = 'copepod'
#Annotation file for images
annFile = "../tmp2-coco.json"
#dump dict looks like a java object, but it really a python dictionary we're going to be saving at the end
# dump_dict = {"image": [], "categories": [], "annotations": [], }
coco=COCO(annFile) #load via cocoapi
cats = coco.loadCats(coco.getCatIds()) #get categorys to add to each json out
# cats_names = [i["name"] for i in cats]
# dump_dict["categories"] = cats

print(f"Categories: {[i["name"] for i in cats]}")
for cat in cats:
    cat_name = cat["name"]
    base_out = os.path.join("/home/sam_woodman_noaa_gov", "Dir0002-out", cat_name)
    os.makedirs(base_out, exist_ok=True)
    
    catIds = coco.getCatIds(catNms=[cat_name]) #use to pull cat ID from cat
    imgIds = coco.getImgIds(catIds=catIds ) #get all images with the specified category above in

    print(f"For category {cat_name}, there are {len(imgIds)} images")
    for img_idx in imgIds:
        # Get and read in the current image
        img = coco.loadImgs(img_idx)[0]
        I = cv2.imread(img['file_name'])
        # I

        # Get the current annotation(s)
        annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)

        for i in anns:
            crop_bbox = anns[0]["bbox"]
            #make cropped image
            cropim = I[math.floor(crop_bbox[1]):math.ceil(crop_bbox[1] + crop_bbox[3]), 
                    math.floor(crop_bbox[0]):math.ceil(crop_bbox[0]+crop_bbox[2])]
            
            #save cropped image
            # TODO: file name. This overwrites. What is our standard?
            path_out = os.path.join(base_out, os.path.basename(img["file_name"]))
            cv2.imwrite(path_out, cropim)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
For category copepod, there are [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 16, 19, 20, 21, 22, 23, 25, 26, 28, 29, 30, 31, 32, 39, 40, 44, 48, 50, 51, 53, 58, 62, 64, 65, 68, 70, 71, 72, 74, 76, 81, 84, 85, 86, 87, 88, 90, 91, 94, 103, 109, 110, 111, 119, 121, 124, 125, 128, 129, 131, 132, 133, 136, 137, 138, 139, 142, 146, 147, 148, 149, 150, 151, 154, 157, 158, 160, 161, 163, 165, 167, 168, 169, 171, 172, 175, 176, 177, 178, 182, 183, 185, 188, 190, 191, 193, 194, 199, 201, 203, 204, 205, 206, 207, 208, 209, 210, 213, 214, 215, 216, 218, 219, 221, 222, 224, 225, 226, 229, 230, 231, 234, 235, 242, 243, 244, 245, 246, 247, 251, 252, 254, 256, 257, 258] images
For category larvacean, there are [130, 134, 138, 11, 14, 142, 144, 18, 155, 174, 49, 179, 53, 54, 71, 72, 77, 86, 89, 90, 217, 95, 96, 99, 101, 102, 103, 110, 111, 238, 239, 115, 118] images
For category ctenophore, there are [256, 198, 13, 86, 152, 156] 

In [ ]:
### Sam exp - individual files
q = 2

# Get and read in the current image
img = coco.loadImgs(imgIds[q])[0]
I = cv2.imread(img['file_name'])
I

# Get the current annotation(s)
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)

crop_bbox = anns[0]["bbox"]
#make cropped image
cropim = I[math.floor(crop_bbox[1]):math.ceil(crop_bbox[1] + crop_bbox[3]), math.floor(crop_bbox[0]):math.ceil(crop_bbox[0]+crop_bbox[2])]
#save cropped image
cv2.imwrite(os.path.join("/home/sam_woodman_noaa_gov", os.path.basename(img["file_name"])), 
            cropim)

In [ ]:
for q in range(0, len(imgIds)):#q is an image with interesting category in it
    img = coco.loadImgs(imgIds[q])[0]#load image name via cocoapi
    I = cv2.imread(os.path.join(working_dir, img['file_name'])) #load the image with cv2
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None) #Get image worm annotations
    anns = coco.loadAnns(annIds)
    embryo_annIds = coco.getAnnIds(imgIds=img['id'], catIds=[1], iscrowd=None) #Get image embryo annotations
    embryo_anns = coco.loadAnns(embryo_annIds)
    for worm_no in range(0, len(anns)):#worm_no is zero indexed #worms in image
        #dump dict looks like a java object, but it really a python dictionary we're going to be saving at the end
        #dump_dict = {"image": [], "categories": [], "annotations": [], }
        #Get the bbox from the worm to use as a crop box
        crop_bbox = anns[worm_no]["bbox"]
        #Get a list of embryos that are inside this crop box 
        embryos_in_worm = [embryo_anns[a] for a in range(0, len(embryo_anns)) if bbox_relation(crop_bbox, embryo_anns[a]['bbox']) == True]
        #make cropped image
        cropim = I[math.floor(crop_bbox[1]):math.ceil(crop_bbox[1] + crop_bbox[3]), math.floor(crop_bbox[0]):math.ceil(crop_bbox[0]+crop_bbox[2])]
        #save cropped image
        file_name_variable = img["file_name"] + "_worm_no_" + str(worm_no)
        cv2.imwrite(working_dir + file_name_variable, cropim)
        for b in range(0, len(embryos_in_worm)): #b is an embryo within the cropped image
            #Transpose embryo seg FIRST ALWAYS!! (function above)
            embryos_in_worm[b]['segmentation'] = seg_transpose(crop_bbox, embryos_in_worm[b]['segmentation'])
            #Transpose embryo bbox (function above - could also calculate from seg)
            embryos_in_worm[b]['bbox'] = bbox_transpose(crop_bbox, embryos_in_worm[b]['bbox'])
            #add new image id(assigning to image after!)
            embryos_in_worm[b]['image_id'] = new_im_id
            #add embryo annotation to dump_dict_annotations
            dump_dict["annotations"].append(embryos_in_worm[b])
        #Same but worm - only one per image probably, so no need for loop.
        worm_transpose_store = anns[worm_no]
        worm_transpose_store["image_id"] = new_im_id
        worm_transpose_store["segmentation"] = seg_transpose(anns[worm_no]["bbox"], worm_transpose_store["segmentation"])
        worm_transpose_store["bbox"] = bbox_transpose(anns[worm_no]["bbox"], anns[worm_no]["bbox"])
        dump_dict["annotations"].append(worm_transpose_store)
        #updating image dictionary for new json
        img_transpose_store = img
        img_transpose_store["height"] = crop_bbox[3]
        img_transpose_store["width"] = crop_bbox[2]
        img_transpose_store["id"] = new_im_id #id = count of worms from 0 - n-1
        img_transpose_store["file_name"] = file_name_variable
        #add image dictionary to dump_dict
        dump_dict["image"].append(img_transpose_store)
        #Update image number
        new_im_id += 1

#Straighten out any annotation duplications due to bbox overlap - duplicate annotations are a problem - deal with this via worm seg!!
for w in range(0, len(dump_dict["annotations"])):
    dump_dict['annotations'][w]["id"] = w
    
#Dump a single coco format json with all cropped images and annotations inside.
with open(working_dir + "worm_by_worm_embryo_test_transposed.json", "w") as outfile:
    json.dump(dump_dict, outfile, indent=4)